# Using Zero shot and Few shot Learning on ChatGroq models

In [86]:
import pandas as pd 
from langchain_groq.chat_models import ChatGroq
import csv

In [87]:
# Groq API and Models 
Groq_Token = "API_Key_here"

groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}

# Zero Shot Learning

In [88]:
features_row = 0
with open('TestDataset_561.csv', 'r') as csv_file:
    reader = csv.reader(csv_file)
    rows = list(reader)

if features_row < len(rows):
    row = rows[features_row][:-2]
    features_string = ', '.join(row)
else:
    print(f"Error in features reading")

test_rows = [10, 50, 70, 90, 120, 150] #STANDING, SITTING, LAYING, WALKING, WALKING_DOWNSTAIRS, WALKING_UPSTAIRS
test_rows_string = []
for test_rown in test_rows:
    with open('TestDataset_561.csv', 'r') as csv_file:
        reader = csv.reader(csv_file)
        rows = list(reader)

    if test_rown < len(rows):
        row = rows[test_rown][:-2]
        test_rown_string = ', '.join(row)
        test_rows_string.append(test_rown_string)
    else:
        print(f"Error in test row reading")

query = f"""
* You are a activity recognition model. 
* Your task is to analyze the given data in form of certain features and classify the activity as 'LAYING', 'SITTING', 'STANDING', 'WALKING', 'WALKING_DOWNSTAIRS' or 'WALKING_UPSTAIRS'. 
* The data is normalized between [-1, 1]
* Provide the activity label and, if necessary, a brief explanation of your reasoning.
* The complete list of features is as follows:
{features_string}


Analyze the following data and predict the activities:
1. {test_rows_string[0]}
2. {test_rows_string[1]}
3. {test_rows_string[2]}
4. {test_rows_string[3]}
5. {test_rows_string[4]}
6. {test_rows_string[5]}
""" 

#Only llama3.1-8b and llama3.1-70b accepted such high token queries
#Both have been used to show a generalized result
model_name = "llama3.1-8b"
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print("Answer from llama3.1-8b: ")
print(answer.content)

model_name = "llama3.1-70b"
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print("\n Answer from llama3.1-70b: ")
print(answer.content)

Answer from llama3.1-8b: 
Based on the given data, I will analyze each data point and predict the activity.

**Data Point 1:**
1.  2.7574570e-001, -1.0371994e-002, -9.9775890e-002, -9.9837313e-001, -9.8693291e-001, -9.9102190e-001, -9.9866291e-001, -9.8713965e-001, -9.9108432e-001, -9.4376125e-001, -5.6428961e-001, -8.1432563e-001,  8.4975268e-001,  6.8922983e-001,  8.4737582e-001, -9.9110378e-001, -9.9999154e-001, -9.9983492e-001, -9.9973485e-001, -9.9859886e-001, -9.8980896e-001, -9.8928972e-001, -7.6858336e-001, -4.3323793e-001, -3.9328675e-001,  3.9409003e-001, -1.7537723e-001,  1.3718364e-001,  1.8434982e-001,  8.3437153e-002, -2.1274865e-002,  3.8584767e-002, -6.5816959e-002,  4.2912522e-001, -2.7979818e-001,  1.5752764e-001, -5.2535204e-002,  1.2899289e-002, -7.8381560e-002,  2.5741515e-001,  9.2417336e-001, -3.1759660e-001,  1.2458390e-001, -9.9800172e-001, -9.8939331e-001, -9.9196835e-001, -9.9791812e-001, -9.8990488e-001, -9.9261060e-001,  8.5020163e-001, -3.3207944e-001,  1.

# Few Shot Learning

In [91]:
features_row = 0
with open('TestDataset_561.csv', 'r') as csv_file:
    reader = csv.reader(csv_file)
    rows = list(reader)

if features_row < len(rows):
    row = rows[features_row][:-2]
    features_string = ', '.join(row)
else:
    print(f"Error in features reading")

#This data will be given as examples for the LLM to learn and give better predictions
train_rows = [[10, 190], [730, 890], [1380, 1540], [1990, 2160], [2670, 2840], [2410, 3590]]
train_labels = ["STANDING", "SITTING", "LAYING", "WALKING", "WALKING_DOWNSTAIRS", "WALKING_UPSTAIRS"]
train_string = ""
for i in range(len(train_rows)):
    with open('Dataset_561.csv', 'r') as csv_file:
        reader = csv.reader(csv_file)
        rows = list(reader)
    for rown in train_rows[i]:
        if (rown < len(rows)):
            row = rows[rown][:-2]
            train_rown_string = ', '.join(row)
            train_string = train_string + '\n' + train_rown_string + ': ' + train_labels[i]
        else:
            print("Error in train row reading")

test_rows = [10, 50, 70, 90, 120, 150] #STANDING, SITTING, LAYING, WALKING, WALKING_DOWNSTAIRS, WALKING_UPSTAIRS
test_rows_string = []
for test_rown in test_rows:
    with open('TestDataset_561.csv', 'r') as csv_file:
        reader = csv.reader(csv_file)
        rows = list(reader)

    if test_rown < len(rows):
        row = rows[test_rown][:-2]
        test_rown_string = ', '.join(row)
        test_rows_string.append(test_rown_string)
    else:
        print(f"Error in test row reading")

query = f"""
* You are a activity recognition model. 
* Your task is to analyze the given data in form of certain features and classify the activity as 'LAYING', 'SITTING', 'STANDING', 'WALKING', 'WALKING_DOWNSTAIRS' or 'WALKING_UPSTAIRS'. 
* The data is normalized between [-1, 1]
* Provide the activity label and, if necessary, a brief explanation of your reasoning.
* The complete list of features is as follows:
{features_string}

A few examples are as follows:
{train_string}

Analyze the following data and predict the activities:
1. {test_rows_string[0]}
2. {test_rows_string[1]}
3. {test_rows_string[2]}
4. {test_rows_string[3]}
5. {test_rows_string[4]}
6. {test_rows_string[5]}
""" 
model_name = "llama3.1-8b"
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print("Answer from llama3.1-8b: ")
print(answer.content)

model_name = "llama3.1-70b"
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print("\n Answer from llama3.1-70b: ")
print(answer.content)

Answer from llama3.1-8b: 
Based on the provided data, the activity labels are:

1.  2.7574570e-001, -1.0371994e-002, -9.9775890e-002, -9.9837313e-001, -9.8693291e-001, -9.9102190e-001, -9.9866291e-001, -9.8713965e-001, -9.9108432e-001, -9.4376125e-001, -5.6428961e-001, -8.1432563e-001,  8.4975268e-001,  6.8922983e-001,  8.4737582e-001, -9.9110378e-001, -9.9999154e-001, -9.9983492e-001, -9.9973485e-001, -9.9859886e-001, -9.8980896e-001, -9.8928972e-001, -7.6858336e-001, -4.3323793e-001, -3.9328675e-001,  3.9409003e-001, -1.7537723e-001,  1.3718364e-001,  1.8434982e-001,  8.3437153e-002, -2.1274865e-002,  3.8584767e-002, -6.5816959e-002,  4.2912522e-001, -2.7979818e-001,  1.5752764e-001, -5.2535204e-002,  1.2899289e-002, -7.8381560e-002,  2.5741515e-001,  9.2417336e-001, -3.1759660e-001,  1.2458390e-001, -9.9800172e-001, -9.8939331e-001, -9.9196835e-001, -9.9791812e-001, -9.8990488e-001, -9.9261060e-001,  8.5020163e-001, -3.3207944e-001,  1.1805617e-001,  9.4442809e-001, -2.8847057e-001,